In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import pytz
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [3]:
data=pd.read_csv(r'C:\Users\Facundo Toneguzzo\Scrapper_Portal\data\raw\2024\202407.csv')

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8699 entries, 0 to 8698
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   url                8699 non-null   object 
 1   Name of the flat   8699 non-null   object 
 2   Value              8699 non-null   int64  
 3   Currency           8699 non-null   object 
 4   General Expenses   7773 non-null   float64
 5   Size of the flat   8645 non-null   float64
 6   Bedrooms           8673 non-null   float64
 7   Bathrooms          8681 non-null   float64
 8   Seller             8699 non-null   object 
 9   metraje            8645 non-null   float64
 10  sup_terraza        4761 non-null   float64
 11  sup_util           8589 non-null   float64
 12  ambientes          4020 non-null   float64
 13  dormitorios        8298 non-null   float64
 14  banos              6572 non-null   float64
 15  estacionamiento    5656 non-null   float64
 16  bodegas            3979 

In [6]:
map_attrib=[column for column in data.columns if data[column].dtype=='object']

In [30]:
data['orientacion'].value_counts().to_dict()

{'Nor-Oriente': 300,
 'SO': 285,
 '-': 280,
 'Nor-Poniente': 276,
 'N': 266,
 'O': 256,
 'SP': 241,
 'P': 222,
 'NP': 214,
 'Sur-Oriente': 199,
 'Oriente': 172,
 'Norte': 133,
 'S': 127,
 'NOSP': 123,
 'Sur': 107,
 'Sur-Poniente': 72,
 'Todas': 65,
 'Nor-Orienteff': 54,
 'Poniente': 46,
 'NOP': 45,
 'NorPoniente-Sur': 31,
 'NOS': 24,
 'Todas Las Anteriores': 20,
 'Norte-sur': 18,
 'Sur-Oriente-Poniente': 18,
 'OSP': 8,
 'NORORIENTE': 7,
 'PON-SUR': 6,
 'NorOriente-Sur': 6,
 'Oriente Sur': 5,
 'Nor-poniente': 5,
 'Sur-oriente': 5,
 'Nor / Oriente / Poniente': 4,
 'NorOriente-Poniente': 4,
 'Nor-Oriente-Poniente': 3,
 'Oriente-SurPoniente': 3,
 'NPS': 3,
 'Norponiente': 2,
 'Nor-oriente': 2,
 'SUR': 2,
 '1': 1,
 'ORIENTE': 1,
 'Nor-Oriente-Sur': 1}

In [8]:
orientacion={'N':['norte','n'],
'NO':['nor-oriente','nor-orienteff','nororiente','no','on'],
'NP':['norponiente','nor-poniente','np','pn'],
'S':['sur','s'],
'SO':['sur-oriente','oriente sur','so','os'],
'SP':['sur-poniente','pon-sur','sp','ps'],
'O':['oriente','o'],
'P':['poniente','p'],
'NS':['norte-sur','ns','sn'],
'OP':['oriente-poniente','op','po'],
'NOP':['nororiente-poniente','nor-oriente-poniente','nor / oriente / poniente','nop','npo','pno','pon','opn','onp'],
'SOP':['sur-oriente-poniente','osp','oriente-surponiente'],
'NSO':['nororiente-sur','nos','nor-oriente-sur'],
'NSP':['norponiente-sur','nps'],
'NOSP':['todas','todas las anteriores','nosp'],
np.nan:['1','-'],    
}

In [9]:
def map_orient(value):
    if pd.isna(value):
        return np.nan
    value_lower = str(value).lower()
    for categoria, keywords in orientacion.items():
        if value_lower in keywords: 
            return categoria
    return value

In [10]:
data['new_orient']=data['orientacion'].map(map_orient)

In [11]:
data['new_orient'].unique()

array([nan, 'S', 'SO', 'N', 'SP', 'P', 'NP', 'O', 'NO', 'NOSP', 'NS',
       'NSO', 'NOP', 'SOP', 'NSP'], dtype=object)

In [12]:
depa={'Departamento':['departamento','habitacional','completo'],
    'Piso':['piso'],
    'Semi Piso':['semi piso','semi-piso'],
    'Clasico':['clásico','clasico','tradicional','cocina tradicional'],
    'Moderno':['moderno','cocina incorporada'],
    'Monoambiente':['monoambiente','un ambiente','solo un ambiente'],
    'Penthouse':['penthhouse','penhouse','penthouse','penthouse duplex'],
    'Standar':['standar','estándar'],
    'Normal':['normal'],
    'Duplex':['duplex','dúplex','dúpex'],
    'Loft':['loft'],
    'Triplex':['triplex','tríplex'],
    'Oficina':['apto oficina','oficina'],
    'Estudio':['studio','estudio'],
    np.nan:['-']
}

In [13]:
def map_depa(value):
    if pd.isna(value):
        return np.nan
    value_lower = str(value).lower()
    for categoria, keywords in depa.items():
        if value_lower in keywords: 
            return categoria
    return str('review_'+value)

In [14]:
data['map_depa']=data['tipo_depa'].map(map_depa)

In [15]:
data['map_depa'].unique()

array(['Departamento', nan, 'Piso', 'Semi Piso', 'Clasico', 'Penthouse',
       'Duplex', 'Monoambiente', 'Moderno', 'Normal', 'Triplex',
       'Standar', 'Loft', 'Oficina', 'Estudio'], dtype=object)

In [16]:
import re

def extract_flat_id(url):
    # Utilizar una expresión regular para extraer el ID del flat
    match = re.search(r"(MLC-\d+)", url)
    if match:
        return match.group(1)
    return np.nan


In [17]:
columns_to_check = [
    'General Expenses', 'Size of the flat', 'Bedrooms', 'Bathrooms', 'Seller',
    'metraje', 'sup_terraza', 'sup_util', 'ambientes', 'dormitorios', 'banos',
    'estacionamiento', 'bodegas', 'piso_unidad', 'cant_pisos', 'dept_piso',
    'antiguedad', 'tipo_depa', 'orientacion'
]

# Función para calcular la tasa de calidad
def calculate_quality_rate(row, columns):
    total_columns = len(columns)
    non_nan_count = row[columns].notna().sum()
    quality_rate = non_nan_count / total_columns
    return quality_rate


In [18]:
data['flat_id']=data['url'].map(extract_flat_id)
data['quality_rate'] = data.apply(calculate_quality_rate, columns=columns_to_check, axis=1)


In [19]:
data.columns

Index(['url', 'Name of the flat', 'Value', 'Currency', 'General Expenses',
       'Size of the flat', 'Bedrooms', 'Bathrooms', 'Seller', 'metraje',
       'sup_terraza', 'sup_util', 'ambientes', 'dormitorios', 'banos',
       'estacionamiento', 'bodegas', 'piso_unidad', 'cant_pisos', 'dept_piso',
       'antiguedad', 'tipo_depa', 'orientacion', 'Calle', 'Barrio', 'Comuna',
       'Ciudad', 'Dirección', 'Fecha_Publicacion', 'Description', 'new_orient',
       'map_depa', 'flat_id', 'quality_rate'],
      dtype='object')

In [20]:
data.head(4)

,url,Name of the flat,Value,Currency,General Expenses,Size of the flat,Bedrooms,Bathrooms,Seller,metraje,sup_terraza,sup_util,ambientes,dormitorios,banos,estacionamiento,bodegas,piso_unidad,cant_pisos,dept_piso,antiguedad,tipo_depa,orientacion,Calle,Barrio,Comuna,Ciudad,Dirección,Fecha_Publicacion,Description,new_orient,map_depa,flat_id,quality_rate
0,https://www.portalinmobiliario.com/MLC-2555490...,Dúplex Con Jardín I Espoz I Amplio Y Remodelado,24500,UF,440000.0,380.0,3.0,4.0,Vitacura Brokers,380.0,134.0,246.0,NaN,3.0,4.0,3.0,1.0,2.0,2.0,3.0,7.0,Departamento,NaN,Espoz 4000,Vitacura,Chile,Parque Bicentenario,"Espoz 4000, Vitacura, Chile, Parque Bicentenar...",2024-07-06,"Amplio e impecable Dúplex con jardín, en exclu...",NaN,Departamento,MLC-2555490854,0.894737
1,https://www.portalinmobiliario.com/MLC-2555757...,Tobalaba C_421813,4100,UF,45000.0,42.0,1.0,1.0,identidad verificada,42.0,NaN,37.0,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Metro Bilbao,Providencia,RM (Metropolitana),NaN,"Metro Bilbao, Providencia, RM (Metropolitana)",2024-07-06,Oportunidad Inversionistas!! esta arrenndado e...,NaN,NaN,MLC-2555757470,0.473684
2,https://www.portalinmobiliario.com/MLC-2555722...,Venta Depto Providencia 2d+2b+1est+1bod 80m2 H...,8000,UF,160000.0,80.0,2.0,2.0,identidad verificada,80.0,5.0,75.0,NaN,2.0,2.0,1.0,1.0,NaN,12.0,NaN,6.0,Departamento,S,Holanda 300 - 600,Providencia,Chile,Metro Tobalaba - Mall Costanera,"Holanda 300 - 600, Providencia, Chile, Metro T...",2024-07-06,Cuenta con:- 75m2 útiles y 5m2 de terraza con ...,S,Departamento,MLC-2555722768,0.842105
3,https://www.portalinmobiliario.com/MLC-2555742...,¡en Oportunidad! Amplio Dpto En Venta I Barrio...,9190,UF,220000.0,95.0,2.0,2.0,identidad verificada,95.0,14.0,81.0,NaN,2.0,2.0,1.0,1.0,10.0,11.0,6.0,5.0,Departamento,SO,Avenida Hernando De Aguirre 1191,Providencia,Chile,Las Lilas,"Avenida Hernando De Aguirre 1191, Providencia,...",2024-07-06,"Precioso departamento en venta, ubicado a solo...",SO,Departamento,MLC-2555742704,0.947368


In [21]:
def update_missing_pairs(df):
    """Update missing values in specified pairs of columns."""
    # Update 'Bedrooms' and 'dormitorios'
    df['Bedrooms'] = df['Bedrooms'].fillna(df['dormitorios'])
    df['dormitorios'] = df['dormitorios'].fillna(df['Bedrooms'])
    
    # Update 'Bathrooms' and 'banos'
    df['Bathrooms'] = df['Bathrooms'].fillna(df['banos'])
    df['banos'] = df['banos'].fillna(df['Bathrooms'])
    
    # Update 'Size of the flat' and 'metraje'
    df['Size of the flat'] = df['Size of the flat'].fillna(df['metraje'])
    df['metraje'] = df['metraje'].fillna(df['Size of the flat'])
    
    return df

In [22]:
def update_surface_areas(df):
    """Update metraje, sup_util, and sup_terraza columns based on given logic."""
    # Calculate 'metraje' if it is null
    df['metraje'] = df.apply(
        lambda row: row['sup_util'] + row['sup_terraza'] if pd.isnull(row['metraje']) and not pd.isnull(row['sup_util']) and not pd.isnull(row['sup_terraza']) else row['metraje'], 
        axis=1
    )

    # Calculate 'sup_util' if it is null
    df['sup_util'] = df.apply(
        lambda row: row['metraje'] - row['sup_terraza'] if pd.isnull(row['sup_util']) and not pd.isnull(row['metraje']) and not pd.isnull(row['sup_terraza']) else row['sup_util'], 
        axis=1
    )   

    # Calculate 'sup_terraza' if it is null
    df['sup_terraza'] = df.apply(
        lambda row: row['metraje'] - row['sup_util'] if pd.isnull(row['sup_terraza']) and not pd.isnull(row['metraje']) and not pd.isnull(row['sup_util']) else row['sup_terraza'], 
        axis=1
    )
    
    return df

In [23]:
data_clean=update_surface_areas(update_missing_pairs(data))

In [24]:
data_clean.head(4)

,url,Name of the flat,Value,Currency,General Expenses,Size of the flat,Bedrooms,Bathrooms,Seller,metraje,sup_terraza,sup_util,ambientes,dormitorios,banos,estacionamiento,bodegas,piso_unidad,cant_pisos,dept_piso,antiguedad,tipo_depa,orientacion,Calle,Barrio,Comuna,Ciudad,Dirección,Fecha_Publicacion,Description,new_orient,map_depa,flat_id,quality_rate
0,https://www.portalinmobiliario.com/MLC-2555490...,Dúplex Con Jardín I Espoz I Amplio Y Remodelado,24500,UF,440000.0,380.0,3.0,4.0,Vitacura Brokers,380.0,134.0,246.0,NaN,3.0,4.0,3.0,1.0,2.0,2.0,3.0,7.0,Departamento,NaN,Espoz 4000,Vitacura,Chile,Parque Bicentenario,"Espoz 4000, Vitacura, Chile, Parque Bicentenar...",2024-07-06,"Amplio e impecable Dúplex con jardín, en exclu...",NaN,Departamento,MLC-2555490854,0.894737
1,https://www.portalinmobiliario.com/MLC-2555757...,Tobalaba C_421813,4100,UF,45000.0,42.0,1.0,1.0,identidad verificada,42.0,5.0,37.0,3.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Metro Bilbao,Providencia,RM (Metropolitana),NaN,"Metro Bilbao, Providencia, RM (Metropolitana)",2024-07-06,Oportunidad Inversionistas!! esta arrenndado e...,NaN,NaN,MLC-2555757470,0.473684
2,https://www.portalinmobiliario.com/MLC-2555722...,Venta Depto Providencia 2d+2b+1est+1bod 80m2 H...,8000,UF,160000.0,80.0,2.0,2.0,identidad verificada,80.0,5.0,75.0,NaN,2.0,2.0,1.0,1.0,NaN,12.0,NaN,6.0,Departamento,S,Holanda 300 - 600,Providencia,Chile,Metro Tobalaba - Mall Costanera,"Holanda 300 - 600, Providencia, Chile, Metro T...",2024-07-06,Cuenta con:- 75m2 útiles y 5m2 de terraza con ...,S,Departamento,MLC-2555722768,0.842105
3,https://www.portalinmobiliario.com/MLC-2555742...,¡en Oportunidad! Amplio Dpto En Venta I Barrio...,9190,UF,220000.0,95.0,2.0,2.0,identidad verificada,95.0,14.0,81.0,NaN,2.0,2.0,1.0,1.0,10.0,11.0,6.0,5.0,Departamento,SO,Avenida Hernando De Aguirre 1191,Providencia,Chile,Las Lilas,"Avenida Hernando De Aguirre 1191, Providencia,...",2024-07-06,"Precioso departamento en venta, ubicado a solo...",SO,Departamento,MLC-2555742704,0.947368


In [25]:
data_clean['flat_id']=data_clean['url'].map(extract_flat_id)
data_clean['quality_rate'] = data_clean.apply(calculate_quality_rate, columns=columns_to_check, axis=1)


In [26]:
def update_antiguedad(row):
    """Update 'antiguedad' column based on given rules."""
    current_year = datetime.now(pytz.timezone('America/Santiago')).year
    
    antiguedad_value = row['antiguedad']
    
    # If antiguedad is NaN, empty, or None, do nothing
    if pd.isnull(antiguedad_value) or antiguedad_value == '':
        return antiguedad_value
    
    try:
        value = int(antiguedad_value)
    except ValueError:
        return np.nan

    # Replace values less than 0 or greater than 2030 with NaN
    if value < 0 or value > 2030:
        return np.nan
    
    # If value has more than 2 digits, calculate the age
    if len(str(value)) > 2:
        return current_year - value
    
    return value

In [27]:
# Apply the function to the 'antiguedad' column
data_clean['antiguedad'] = data_clean.apply(update_antiguedad, axis=1)

In [28]:
pd.set_option('display.max_rows',1000)

In [29]:
data_clean.sort_values(by='flat_id',ascending=False)

,url,Name of the flat,Value,Currency,General Expenses,Size of the flat,Bedrooms,Bathrooms,Seller,metraje,sup_terraza,sup_util,ambientes,dormitorios,banos,estacionamiento,bodegas,piso_unidad,cant_pisos,dept_piso,antiguedad,tipo_depa,orientacion,Calle,Barrio,Comuna,Ciudad,Dirección,Fecha_Publicacion,Description,new_orient,map_depa,flat_id,quality_rate
8620,https://www.portalinmobiliario.com/MLC-2594880...,"Se Vende Bello Dpto 1h,1b, Bodega/estcnmto, En...",5150,UF,65000.0,49.0,1.0,1.0,identidad verificada,49.0,9.0,40.0,NaN,1.0,1.0,1.0,NaN,7.0,8.0,NaN,0.0,-,-,Map - Excelente Oportunidad Se Vende Bello Dpt...,1b,Estacionamiento Y Bodega,Providencia,Map - Excelente Oportunidad Se Vende Bello Dpt...,2024-07-19,Coordine su visita con Miguel Angel Aguilera P...,NaN,NaN,MLC-2594880428,0.842105
8677,https://www.portalinmobiliario.com/MLC-2594878...,Lyon/carlos Antúnez C_421407,6400,UF,200000.0,68.0,2.0,2.0,identidad verificada,68.0,8.0,60.0,4.0,2.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Los Leones,Providencia,RM (Metropolitana),NaN,"Los Leones, Providencia, RM (Metropolitana)",2024-07-19,"ACOGEDOR Y COMODO DEPARTAMENTO, SECTOR RESIDE...",NaN,NaN,MLC-2594878562,0.631579
8651,https://www.portalinmobiliario.com/MLC-2594868...,Reserva Con 100.000 Duplex Y Rooftop Sin Comis...,17123,UF,280000.0,138.0,3.0,3.0,identidad verificada,138.0,16.0,84.0,3.0,3.0,3.0,1.0,1.0,7.0,NaN,NaN,0.0,Semi piso,N,Reserva Con 100.000 Duplex Y Rooftop Sin Comision,Campus Oriente,Providencia,RM (Metropolitana),Reserva Con 100.000 Duplex Y Rooftop Sin Comis...,2024-07-19,BrokerPartner vende impecable departamento en ...,N,Semi Piso,MLC-2594868848,0.894737
8669,https://www.portalinmobiliario.com/MLC-2594852...,Colegio San Ignacio C_422888,10600,UF,250000.0,113.0,2.0,2.0,identidad verificada,113.0,20.0,93.0,4.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Las Lilas,Providencia,RM (Metropolitana),NaN,"Las Lilas, Providencia, RM (Metropolitana)",2024-07-19,No dejes escapar esta gran e increible oportun...,NaN,NaN,MLC-2594852642,0.578947
8686,https://www.portalinmobiliario.com/MLC-2594852...,Departamento Aguilucho / Suecia Id: 97779,8500,UF,NaN,76.0,2.0,2.0,Houm,76.0,3.0,73.0,2.0,2.0,2.0,1.0,1.0,NaN,NaN,NaN,0.0,Departamento,NaN,Aguilucho / Suecia,Campus Oriente,Providencia,RM (Metropolitana),"Aguilucho / Suecia, Campus Oriente, Providenci...",2024-07-19,[VEN] ID HOUM: 97779En Houm te lo hacemos más ...,NaN,Departamento,MLC-2594852464,0.736842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
591,https://www.portalinmobiliario.com/MLC-1500315...,Departamento En Venta De 3 Dorm. En Vitacura,12800,UF,180000.0,141.0,3.0,3.0,identidad verificada,141.0,11.0,130.0,3.0,3.0,3.0,2.0,1.0,NaN,3.0,2.0,40.0,Semi piso,NaN,Departamento En Venta De 3 Dorm. En Vitacura,Parque Bicentenario,Vitacura,RM (Metropolitana),"Departamento En Venta De 3 Dorm. En Vitacura, ...",2024-07-07,Se vende espacioso departamento en la exclusiv...,NaN,Semi Piso,MLC-1500315273,0.894737
562,https://www.portalinmobiliario.com/MLC-1500315...,Departamento En Venta De 3 Dorm. En Vitacura,12800,UF,180000.0,141.0,3.0,3.0,identidad verificada,141.0,11.0,130.0,3.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Departamento En Venta De 3 Dorm. En Vitacura,Parque Bicentenario,Vitacura,RM (Metropolitana),"Departamento En Venta De 3 Dorm. En Vitacura, ...",2024-07-07,Se vende espacioso departamento en la exclusiv...,NaN,NaN,MLC-1500315273,0.578947
9,https://www.portalinmobiliario.com/MLC-1500315...,Departamento En Venta De 3 Dorm. En Vitacura,12800,UF,180000.0,141.0,3.0,3.0,identidad verificada,141.0,11.0,130.0,3.0,3.0,3.0,2.0,1.0,NaN,3.0,2.0,40.0,Semi piso,NaN,Departamento En Venta De 3 Dorm. En Vitacura,Parque Bicentenario,Vitacura,RM (Metropolitana),"Departamento En Venta De 3 Dorm. En Vitacura, ...",2024-07-06,Se vende espacioso departamento en la exclusiv...,NaN,Semi Piso,MLC-1500315273,0.894737
1177,https://www.portalinmobiliario.com/MLC-150031